# Lokális képosztályozók 

Olvasd el az [elődás olvasóleckéjét](http://inf.u-szeged.hu/~rfarkas/ML20/image.html)!

## MNIST digits adatbázis

A képosztályozási feladatnál az egyedek képek, melyeket előre definálit kategórák/osztályok közül az egyikbe kell besorolnunk. A híres [MNIST digits](https://www.kaggle.com/c/digit-recognizer) adatbázisban kézzel írt számjegyek szerepelnek (10 kategóriás osztályozási feladat)

In [ ]:
 from sklearn.datasets import load_digits
 digits = load_digits()
 digits

Az MNIST digits adatbázs már nagyon szépen előfeldolgozott (egységes méret, nyújtás, színskála stb.) képeket tartalmaz:

In [ ]:
print(digits.images.shape)
print(digits.data.shape)

In [ ]:
#ha meg akarjuk jeleníteni a képeket a notebookban
import matplotlib.pyplot as plt
plt.imshow(digits.images[0])

In [ ]:
plt.imshow(digits.images[0], cmap=plt.cm.gray_r)

In [ ]:
digits.target[0]

In [ ]:
import pandas as pd
pd.value_counts(digits.target).sort_index()

A legegyszerűbb jellemzőleírása képeknek, amikor az egyedeket leírjuk a pixeljeivel. Az MNIST digit adatbázisban a `data` vektor közvetlenül használható jellemzővektorként (a 64 pixel a 64 jellemző, a szürkeárnyalat intenzitás a jellemző értéke 0-255)

In [ ]:
from sklearn.model_selection import train_test_split
trainFeatures, testFeatures, trainLables, testLables = train_test_split(digits.data, 
                                                                        digits.target, 
                                                                        test_size=0.33, 
                                                                        random_state=42)

Bármilyen osztályozó algoritmust használhatnánk, de most a kicsi tanító adatbázis, és azért mert a pixelek feletti hasonlósági metrika itt nagyon jó metrika, ezért használjuk a '**k legközelebbi szomszéd**' (k nearest neighboor, kNN) osztályozót ([lásd](http://inf.u-szeged.hu/~rfarkas/ML20/image.html)).

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=15) # n_neighbors a k érték (szomszédok száma)


In [ ]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()

In [ ]:
from sklearn.metrics import accuracy_score
model.fit(trainFeatures, trainLables)
prediction = model.predict(testFeatures)
accuracy_score(y_true=testLables, y_pred=prediction)

## Macska-kutya képosztályozási feladat

Alább a '[macskák vs kutyák](https://www.kaggle.com/c/dogs-vs-cats/)' képosztályozási feladatra adunk gépi tanulási megoldást. Itt bináris osztályozót kell építeni, ami vagy a *macska* vagy a *kutya* osztályba sorol be egy képet. A [tanító adatbázis](https://www.kaggle.com/c/dogs-vs-cats/data) 25ezer képet tartalmaz, amelyet emberek *macska* vagy *kutya* címkével láttak el.

(A [kaggle](http://www.kaggle.com) gépi tanulási (verseny)feladatok tárháza, futtatható notebookokkal és élő közösséggel akik megoldásokat is megtárgyalnak. Gyakorlásra/versenyzésre ajánlom mindenkinek...)

In [ ]:
#letöltjük a címkézett képfájlokat
import os
import urllib.request
#az eredeti kaggle adatbázisból 100 képet kiemeltem, hogy pillanatok alatt fusson a kód
#természetesen ha pontosabb rendszert szeretnénk akkor érdemes nagyobb képi adatbázissal dolgozni
url = 'https://github.com/rfarkas/student_data/raw/main/images/100imgs.zip'
urllib.request.urlretrieve(url,'t.zip') #a t.zip a temporálisan notebookhoz rendelt tárhelyre kerül a Google felhőjében

#kitömörítjük a zip tartalmát
import zipfile
zipfile.ZipFile('t.zip').extractall('tmp_imgs')

## Képek előfeldolgozása
Ahhoz, hogy a kép egyedeket gépi tanulási feladatban használni tudjuk, le kell írni őket jellemzőkkel. Ehhez általában első lépésként normalizálni szükséges a képadatbázisokat, például egyenméretre kell alakítani a képeket.

A képek (és videók) előfeldolgozásának - így normalizálás és jellemzőkinyerés - legelterjedtebb eszköze az [OpenCV](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html). 

In [ ]:
import cv2 as cv #OpenCV

Az OpenCV a színes képeket 3 dimenziós tömbökben (tenzorban) tárolja. Az első két dimenzió egy pixel x és y koordinátáival indexelhető és minden pixelhez 3 színcsatorna érték tartozik:

In [ ]:
image = cv.imread('tmp_imgs/cat.8000.jpg') # egy JPEG fájl a temporális könyvtárból
image

In [ ]:
image.shape

In [ ]:
#ha meg akarjuk jeleníteni a képeket a notebookban
from matplotlib import pyplot as plt
plt.imshow(image)

Kicsit furák a színek...

Azért mert az OpenCV blue, green, red sorrendben tárolja a színcsatornákat, míg a notebookunk RGB sorrendet feltételez ([BGR2RGB](https://giusedroid.blogspot.hu/2015/04/blog-post.html) részletesebben). Egyszerűen átkonvertálhatjuk RGBbe:

In [ ]:
plt.imshow(cv.cvtColor(image,cv.COLOR_BGR2RGB))

Használjuk itt is jellemzőnek a kép pixeleinek színét! Ehhez a jellemzőkészlethez viszont elengedhetetlen, hogy a képek egyenméretűek legyenek.

In [ ]:
image.shape #egy kép méretei: magasság pixelszám, szélesség pixelszám, színcsatornák száma

In [ ]:
# több módszer is van egyenméretezésre (például levágás), de most csak tömörítünk:
image_small = cv.resize(image,(32,32)) # 32 x 32 méretűre konvertáljuk a képet
plt.imshow(cv.cvtColor(image_small,cv.COLOR_BGR2RGB))

Emberi szem számára a minőség sokat romlik, de így egy pixel még mindig leírja, hogy a kép azon területén milyen színek fordulnak elő, ezzel közelítve azt, hogy a kép adott területén mi fordul elő...

In [ ]:
os.listdir('tmp_imgs')

In [ ]:
rawImages = [] # a lista minden eleme egy eredeti kép
features = [] # a lista minden eleme egy kép pixelvektora
labels = [] # a lista minden eleme a kép címkéje ('cat' vagy 'dog')

In [ ]:
for f in os.listdir('tmp_imgs'): # könyvtárbejárás
  image = cv.imread('tmp_imgs/'+f) # kép beolvasása
  label = f.split(os.path.sep)[-1].split(".")[0] # a fájlnév megadja most a címkét (vagy cat vagy dog az első '.' előtt fájlnév)
  
  #flatten a 3D tömbből 1D-t csinál (egymás után fűzi a sorokat)
  pixels = cv.resize(image,(32,32)).flatten() # egyenméretűre hozzuk a képeket! 32 x 32 x 3 = 3072 érték képenként
    
  rawImages.append(image)
  features.append(pixels)
  labels.append(label)

In [ ]:
pixels.shape

In [ ]:
# ellenőrzés:
print("lengths:", len(labels), len(rawImages), len(features))
print("# features:", len(features[0]))
pd.value_counts(labels)

Elkülönítünk egy teszt halmazt, amit a tanítás során nem láthatunk/használhatunk (hold out set). A 100db képünkből, használjuk az első 80-at tanító adatbázisként és az utolsó 20-on értékeljünk ki. Egy címkézett adatbázis ilyen szétosztását **train-test split**nek hívjuk.

In [ ]:
trainLables = labels[:81]
testLables  = labels[81:]
trainFeatures = features[:81]
testFeatures  = features[81:]
trainImages = rawImages[:81]
testImages  = rawImages[81:]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5) # n_neighbors a k érték (szomszédok száma)
model.fit(trainFeatures, trainLables) # tanítás

In [ ]:
from sklearn.metrics import accuracy_score
prediction = model.predict(testFeatures)
accuracy_score(y_true=testLables, y_pred=prediction)

Mennyire jó ez az accuracy?

Csak úgy tudjuk megítélni ha valamilyen nagyon egyszerű döntési szabályhoz ([**baseline**](http://inf.u-szeged.hu/~rfarkas/ML20/baseline.html) vagy dummy) hasonlítjuk. Ha egyszerű döntési szabályoknál jobb eredményt érünk el, akkor mondhatjuk, hogy tanult valamit az algoritmus.

In [ ]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='most_frequent',random_state=0) # most frequent class baseline
dummy.fit(trainFeatures, trainLables)
dummy.score(testFeatures, testLables)


Azt állíthatjuk, hogy 
*   ezen a 80darab képes tanító adatbázison és
*   ezzel a jellemzőleírással és
*   az így paraméterezett kNN tanulóval

nem tudunk tanulni. A tanító adatbázis méretének növelésén túl, másik osztályozó választása lehet a megoldás. A kNN legnagyobb hátránya, hogy csak jól definált hasonlósági függvény mellett működik jól. Alapértelmezett esetben a jellemzővektorok súlyozatlan euklideszi távolságát használja.

Mindig próbáljuk meg megérteni mit csinál a model! A kNN-nél egyszerűen ki lehet listázni, hogy melyik képhez melyek a leghasonlóbbak (legközelebbi szomszédok).

In [ ]:
# első tesztkép:
plt.imshow(cv.cvtColor(testImages[0],cv.COLOR_BGR2RGB))

In [ ]:
nn = model.kneighbors(testFeatures[0:1], return_distance=False) # az első tesztképhez talált 5 legközelebbi szomszéd (a tanító adatbázisból) indexe
nn # neighborhood

In [ ]:
# a neighborhood megjelenítése
fig, imgs = plt.subplots(1,5)
for k in range(5):
  train_idx = nn[0][k]
  imgs[k].imshow(cv.cvtColor(trainImages[train_idx],cv.COLOR_BGR2RGB))
plt.tight_layout()
plt.show()

Mire tanítottuk a gépet!? Arra, hogy ugyanazon az x,y koordinátán lévő pixelek színei hasonlóak legyenek... 

## Képbeágyazások

Amikor a pixelhasonlóság nem elég jó (mert bonyolult objektumokat kell kategórizálni) akkor jobb jellemző leírást adnak az előre tanított képbeágyazások.

A https://github.com/christiansafka/img2vec a [pytorch](https://towardsdatascience.com/using-predefined-and-pretrained-cnns-in-pytorch-e3447cbe9e3c)-ra épül, és az [ImageNet](http://www.image-net.org/) 1.4 millió képén lett előtanítva.

In [ ]:
!wget https://raw.githubusercontent.com/christiansafka/img2vec/master/img2vec_pytorch/img_to_vec.py

In [ ]:
from img_to_vec import Img2Vec
img2vec = Img2Vec()

In [ ]:
from PIL import Image # PIL típusú képformátumban várja az inputot az img2vec
img = Image.open('tmp_imgs/cat.8000.jpg')
img2vec.get_vec(img)

Minden képet egy 512 dimenziós sűrő vektorként ágyaz be a ResNet-18 előretanított neurális hálózat.

In [ ]:
len(img2vec.get_vec(img))

In [ ]:
# rawImage formátumunkból így tudunk PTL-t gyártani:
img = Image.fromarray(rawImages[0])
len(img2vec.get_vec(img))

In [ ]:
# A képegyedeket most az img2vec beágyazással írjuk le (512 folytonos jellemző)
vecs = []
for img in rawImages:
  vecs.append( img2vec.get_vec( Image.fromarray(img) ))

In [ ]:
trainFeatures = vecs[:81]
testFeatures  = vecs[81:]

Nagyon kevés a tanító példánk, de most már jó hasonlósági metrikánk van (képbeágyazások között számolt vektorhasonlóság vizuális tartalmi hasonlóságot ír le), így a kNN a jó választás:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=13) # n_neighbors a k érték (szomszédok száma)
model.fit(trainFeatures, trainLables) # tanítás

In [ ]:
from sklearn.metrics import accuracy_score
prediction = model.predict(testFeatures)
accuracy_score(y_true=testLables, y_pred=prediction)

In [ ]:
prediction != testLables

Ez nagyon szép :) De a képbeágyazások sem mágikus csodafegyver! Az ImageNet, amin a beágyazás tanítva lett több száz állatfajt tartalmaz ill. a 'kutya vs macska' feladat nagyon könnyű. Ha például arcokat kellene érzelem alapján osztályozni (gyakorló feladat), arra már nem lesz ilyen jó a beágyazás. Ismernünk kell, hogy hogyan készültek a beágyazások, csak így tudjuk megállapítani, hogy az adott célfeladathoz jó lesz-e!

In [ ]:
plt.imshow(cv.cvtColor(testImages[-3],cv.COLOR_BGR2RGB))

In [ ]:
nn = model.kneighbors(testFeatures[-3:-2], return_distance=False)

# Gyakorló feladatok

1. A megismert 100 kutya/macska adatbázison futtasd le az alábbi kísérletet.
*    szürkeárnyalatos képeken
*    64 x 64-re átméretezve
*    train/test 80-20% vágás
*    sztochasztikus gradient descent tanulóval
*    jellemzőkészlet: pixelek

Kérdés, hogy mennyire függ az elért pontosság a train test split véletlenszerűségétől! Ehhez százszor futtasd le, hogy
a 100 képből véletlenszerűen választasz 80 train és 20 test esetet tanítasz a trainen, accuracyt mérsz a teszten.



2. Milyen pontossággal tudjuk egy arcképről azonosítani az érzelmeket?

Hajts végre egy tanító-kiértékelő adatbázis bontással predikciós kísérletet az alábbi adatbázison!

In [ ]:
import pandas as pd
data = pd.read_csv('https://github.com/rfarkas/student_data/raw/main/images/fer2013_1K.csv')

In [ ]:
#48x48 szürke képek egy stringben vannak kódolva. A 11. kép kiolvasása:
import numpy as np
image = np.reshape(np.asarray(data.pixels[11].split(' '), np.uint8), (48,48))
plt.imshow(image, cmap="gray")

---
**Kitekintés:**
Az elmút 10 évben a képosztályozásban mindent visz a Convolutional Neural Network (CNN), ami a deep learning egyik legnagyobb sikersztorija! Ez egy jó [tutorial](http://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/).